![car](car.jpg)

Insurance companies invest a lot of [time and money](https://www.accenture.com/_acnmedia/pdf-84/accenture-machine-leaning-insurance.pdf) into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries insurance it is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

Knowing all of this, On the Road car insurance have requested your services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked you to identify the single feature that results in the best performing model, as measured by accuracy, so they can start with a simple model in production.

They have supplied you with their customer data as a csv file called `car_insurance.csv`, along with a table detailing the column names and descriptions below.



## The dataset

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-15</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code | 
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client | 
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

In [31]:
# Import required modules
import pandas as pd
import numpy as np
from statsmodels.formula.api import logit

# Start coding!

In [32]:
df=pd.read_csv("car_insurance.csv")
df.head()

,id,age,gender,driving_experience,education,income,credit_score,vehicle_ownership,vehicle_year,married,children,postal_code,annual_mileage,vehicle_type,speeding_violations,duis,past_accidents,outcome
0,569520,3,0,0-9y,high school,upper class,0.629027,1.0,after 2015,0.0,1.0,10238,12000.0,sedan,0,0,0,0.0
1,750365,0,1,0-9y,none,poverty,0.357757,0.0,before 2015,0.0,0.0,10238,16000.0,sedan,0,0,0,1.0
2,199901,0,0,0-9y,high school,working class,0.493146,1.0,before 2015,0.0,0.0,10238,11000.0,sedan,0,0,0,0.0
3,478866,0,1,0-9y,university,working class,0.206013,1.0,before 2015,0.0,1.0,32765,11000.0,sedan,0,0,0,0.0
4,731664,1,1,10-19y,none,working class,0.388366,1.0,before 2015,0.0,0.0,32765,12000.0,sedan,2,0,1,1.0


In [33]:
#droping na and keeping safe my original fine
df2=df.drop_duplicates()
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10000 non-null  int64  
 1   age                  10000 non-null  int64  
 2   gender               10000 non-null  int64  
 3   driving_experience   10000 non-null  object 
 4   education            10000 non-null  object 
 5   income               10000 non-null  object 
 6   credit_score         9018 non-null   float64
 7   vehicle_ownership    10000 non-null  float64
 8   vehicle_year         10000 non-null  object 
 9   married              10000 non-null  float64
 10  children             10000 non-null  float64
 11  postal_code          10000 non-null  int64  
 12  annual_mileage       9043 non-null   float64
 13  vehicle_type         10000 non-null  object 
 14  speeding_violations  10000 non-null  int64  
 15  duis                 10000 non-null  

In [34]:
#cleaning data
#1)by first filling na with means 
mean_credit= df2['credit_score'].mean()
df2['credit_score']=df2['credit_score'].fillna(mean_credit)
mean_mil= df2['annual_mileage'].mean()
df2['annual_mileage']=df2['annual_mileage'].fillna(mean_mil)
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10000 non-null  int64  
 1   age                  10000 non-null  int64  
 2   gender               10000 non-null  int64  
 3   driving_experience   10000 non-null  object 
 4   education            10000 non-null  object 
 5   income               10000 non-null  object 
 6   credit_score         10000 non-null  float64
 7   vehicle_ownership    10000 non-null  float64
 8   vehicle_year         10000 non-null  object 
 9   married              10000 non-null  float64
 10  children             10000 non-null  float64
 11  postal_code          10000 non-null  int64  
 12  annual_mileage       10000 non-null  float64
 13  vehicle_type         10000 non-null  object 
 14  speeding_violations  10000 non-null  int64  
 15  duis                 10000 non-null  

In [35]:
print(df['driving_experience'].unique())
print(df['education'].unique())
print(df['income'].unique())
print(df['vehicle_type'].unique())

['0-9y' '10-19y' '20-29y' '30y+']
['high school' 'none' 'university']
['upper class' 'poverty' 'working class' 'middle class']
['sedan' 'sports car']


now that i know the uniques i will try to replace them with numbers so as to run them well with my model


In [36]:
#this function will be useed to replace string with n.o in the drive experience colum
def drive(x):
    if x == '0-9y':
        return 0
    elif x == '10-19y':
        return 1
    elif x == '20-29y' :
        return 2
    else:
        return 3
#this next one will work on the education colums
def educ(x):
    if x == 'none':
        return 0
    elif x == 'high school':
        return 1
    elif x == 'university':
        return 2
    else:
        return 1
#function for the income column
def inc(x):
    if x == 'poverty':
        return 0
    elif x == 'working class':
        return 1
    elif x == 'middle class':
        return 2
    else:
        return 3
#function for the vihicle coumn
def vih(x):
    if x == 'sedan':
        return 0
    else:
        return 1

In [37]:
#calling all the functions now 
df2['driving_experience']= df['driving_experience'].apply(drive)
df2['education']= df['education'].apply(educ)
df2['income']= df['income'].apply(inc)
df2['income']= df['income'].apply(inc)
df2.head()

,id,age,gender,driving_experience,education,income,credit_score,vehicle_ownership,vehicle_year,married,children,postal_code,annual_mileage,vehicle_type,speeding_violations,duis,past_accidents,outcome
0,569520,3,0,0,1,3,0.629027,1.0,after 2015,0.0,1.0,10238,12000.0,sedan,0,0,0,0.0
1,750365,0,1,0,0,0,0.357757,0.0,before 2015,0.0,0.0,10238,16000.0,sedan,0,0,0,1.0
2,199901,0,0,0,1,1,0.493146,1.0,before 2015,0.0,0.0,10238,11000.0,sedan,0,0,0,0.0
3,478866,0,1,0,2,1,0.206013,1.0,before 2015,0.0,1.0,32765,11000.0,sedan,0,0,0,0.0
4,731664,1,1,1,0,1,0.388366,1.0,before 2015,0.0,0.0,32765,12000.0,sedan,2,0,1,1.0


CLEANING IS  DONE NOW WE GET TO RUNIG THE MODEL

In [38]:
from statsmodels.formula.api import logit

In [39]:
models=[]
features = df2.drop(columns=['outcome', 'id']).columns

# Display the list of features
print(features)

Index(['age', 'gender', 'driving_experience', 'education', 'income',
       'credit_score', 'vehicle_ownership', 'vehicle_year', 'married',
       'children', 'postal_code', 'annual_mileage', 'vehicle_type',
       'speeding_violations', 'duis', 'past_accidents'],
      dtype='object')


In [40]:
#creating the formulae list
formulae=[]
for col in features:
    formulae.append("outcome~"+col)
    
print(formulae)

['outcome~age', 'outcome~gender', 'outcome~driving_experience', 'outcome~education', 'outcome~income', 'outcome~credit_score', 'outcome~vehicle_ownership', 'outcome~vehicle_year', 'outcome~married', 'outcome~children', 'outcome~postal_code', 'outcome~annual_mileage', 'outcome~vehicle_type', 'outcome~speeding_violations', 'outcome~duis', 'outcome~past_accidents']


In [41]:
#creating the models
model_names=[]
for col in features:
    mdl_name="mdl_"+col
    model_names.append(mdl_name)
    mdl= logit("outcome~"+col,data=df2).fit()
    models.append(mdl)
print(model_names)
print(models)

Optimization terminated successfully.
         Current function value: 0.511794
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.615951
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.467390
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.603848
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.531580
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.572557
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.552412
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.572668
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.586659
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.595431
  

GETING THE ACCURACY 
SINCE INTHE CON_TABLE TN IS AT [0,0] AND TRU POSITIVE AT [1,1]

In [42]:
accuracy_t=[]
for x in models:
    conf_tabl=x.pred_table()
    ac=(conf_tabl[0,0]+conf_tabl[1,1])/10000
    accuracy_t.append(ac)
df_ac=pd.DataFrame({"model": model_names , "accuracy": accuracy_t})
df_ac.sort_values(by="accuracy",ascending = False)

,model,accuracy
2,mdl_driving_experience,0.7771
0,mdl_age,0.7747
4,mdl_income,0.7425
6,mdl_vehicle_ownership,0.7351
5,mdl_credit_score,0.7054
11,mdl_annual_mileage,0.6904
1,mdl_gender,0.6867
3,mdl_education,0.6867
7,mdl_vehicle_year,0.6867
8,mdl_married,0.6867


AND THE BEST MODEL WAS

In [43]:
best_feature_df=df_ac.sort_values(by="accuracy",ascending = False)
best_feature_df.iloc[[0]]

,model,accuracy
2,mdl_driving_experience,0.7771
